Import module and data

In [4]:
pip install google-cloud-bigquery

   ---------------------------------------- 0.0/244.6 kB ? eta -:--:--
   ----- ---------------------------------- 30.7/244.6 kB 1.3 MB/s eta 0:00:01
   --------------- ------------------------ 92.2/244.6 kB 1.0 MB/s eta 0:00:01
   ----------------------- ---------------- 143.4/244.6 kB 1.2 MB/s eta 0:00:01
   -------------------------------------- - 235.5/244.6 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 244.6/244.6 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/160.1 kB ? eta -:--:--
   ---------------------------------------- 160.1/160.1 kB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/210.8 kB ? eta -:--:--
   --------------------- ------------------ 112.6/210.8 kB 6.4 MB/s eta 0:00:01
   ---------------------------------------- 210.8/210.8 kB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/81.3 kB ? eta -:--:--
   ---------------------------------------- 81.3/81.3 kB 4.7 MB/s eta 0:00:00
 


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\88696\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\88696\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
pip install matplotlib

     ---------------------------------------- 0.0/103.3 kB ? eta -:--:--
     --- ------------------------------------ 10.2/103.3 kB ? eta -:--:--
     ----------- ------------------------- 30.7/103.3 kB 330.3 kB/s eta 0:00:01
     -------------- ---------------------- 41.0/103.3 kB 245.8 kB/s eta 0:00:01
     ------------------------------------ 103.3/103.3 kB 596.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.0 MB ? eta 


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\88696\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\88696\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
import numpy as np
import pandas as pd
from datetime import datetime, date
import os
from google.cloud import bigquery as bq
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
from statistics import mean, stdev, math
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy import stats

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/aaa.json'
client = bq.Client()

query_sa = '''
SELECT *
FROM `aaa`
'''
sa_all = client.query(query_sa).result().to_arrow(create_bqstorage_client=True).to_pandas()

sa_uni_cus = sa_all.drop_duplicates(subset=['Cus_id'])
sa_registered = sa_all[sa_all['product_line'].notnull()]
sa_unregistered = sa_all[sa_all['product_line'].isnull()]
sa_registered_edmconsent = sa_registered[sa_registered['EDMSubscriber']=='yes']

print('Members:',len(sa_all['Cus_id'].unique()))
print('Number of members registered:',len(sa_registered['Cus_id'].unique()))
print('Number of members unregistered:',len(sa_unregistered['Cus_id'].unique()))
print('Number of members registered_EDM subscriber:',len(sa_registered[sa_registered['EDMSubscriber']=='yes']['Cus_id'].unique()))
print('Number of members registered_EDM non-subscriber:',len(sa_registered[sa_registered['EDMSubscriber']=='no']['Cus_id'].unique()))
print('Number of members unregistered_EDM subscriber:',len(sa_unregistered[sa_unregistered['EDMSubscriber']=='yes']['Cus_id'].unique()))
print('Number of members unregistered_EDM subscriber:',len(sa_unregistered[sa_unregistered['EDMSubscriber']=='no']['Cus_id'].unique()))
print('Number of registers:', len(sa_all[sa_all['product_line'].notnull()]))

Building RFM model

In [ ]:
basic_df = pd.DataFrame(columns=['Cus_id', 'Frequency', 'Recently', 'Monetary'])
current_time = datetime.now()
Cus_id_grouped = sa_registered.groupby('Cus_id')
all_data = []
for cus_id, group in Cus_id_grouped:
    frequency = len(group)
    latest_date = max(group['register_date'])
    recently = (current_time - datetime.combine(latest_date, datetime.min.time())).days
    monetary = group['ebs_price'].sum()
    all_data.append({'Cus_id': cus_id, 'Frequency': frequency, 'Recently': recently, 'Monetary': monetary})
basic_df = pd.DataFrame(all_data)
basic_df = basic_df.sort_values('Frequency', ascending=False)

selected_columns = ['Frequency', 'Recently', 'Monetary']
for column in selected_columns:
    transformed_data, lambda_value = stats.yeojohnson(basic_df[column])
    new_column_name = column + '_YJ'
    basic_df[new_column_name] = transformed_data

In [ ]:
def standardize_column(column):
    column_mean = mean(column)
    column_stdev = stdev(column)
    return [(x - column_mean) / column_stdev for x in column]
columns_to_standardize = ['Frequency', 'Recently', 'Monetary', 'Frequency_YJ', 'Recently_YJ', 'Monetary_YJ']
for column in columns_to_standardize:
    basic_df[f'{column}_standardized'] = standardize_column(basic_df[column])

rfm_basic_df = basic_df[['Frequency', 'Recently', 'Monetary']]
rfm_basic_YJ = basic_df[['Frequency_YJ', 'Recently_YJ', 'Monetary_YJ']]
rfm_basic_YJ_str = basic_df[['Frequency_YJ_standardized', 'Recently_YJ_standardized', 'Monetary_YJ_standardized']]

In [ ]:
def label_bars(ax):
    for rect in ax.patches:
        height = rect.get_height()
        total_height = sum([patch.get_height() for patch in ax.patches])
        percentage = height / total_height * 100
        if percentage > 0.0:  # Only label bars with non-zero percentage
            ax.annotate(f'{percentage:.2f}%',
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom',
                        color='black')  # Set label color to black

In [ ]:
for column in rfm_basic_YJ_str:
    plt.figure(figsize=(8, 6))  # Adjust figure size for better visualization
    plt.hist(rfm_basic_YJ_str[column], bins=10, edgecolor='black', color='blue')
    plt.title(f"Histogram of Column '{column}'")
    plt.xlabel("Value")
    plt.ylabel("Users")

    n, bins, patches = plt.hist(rfm_basic_YJ_str[column], bins=10, edgecolor='black')

    label_bars(plt.gca())

    plt.tight_layout()
    plt.show()

In [ ]:
np.random.seed(42)
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, n_init = 10, random_state=42)
basic_df['Segment'] = kmeans.fit_predict(rfm_basic_YJ_str)

In [ ]:
centers = kmeans.cluster_centers_
centers_df = pd.DataFrame(centers, columns=rfm_basic_YJ_str.columns)
cluster_counts = basic_df['Segment'].value_counts()

sorted_cluster_counts = pd.DataFrame(cluster_counts).rename(columns={'Segment': 'Count'})
sorted_cluster_counts.index.name = 'Segment'
sorted_cluster_counts = sorted_cluster_counts.sort_values(by='Count', ascending=False)
sorted_cluster_counts = sorted_cluster_counts.sort_index()
print('各群的樣本數：')
for cluster, count in sorted_cluster_counts.itertuples():
    print(f'Segment {cluster}: {count} 樣本')
print("K-means中心點的位置：")
print(centers_df)
print("各個data到中心點的位置總和：")
kmeans.inertia_

In [ ]:
cluster_means = basic_df.groupby('Segment', as_index=False).mean(numeric_only=True)
frequency_mean = cluster_means['Frequency']
monetary_mean = cluster_means['Monetary']
recently_mean = cluster_means['Recently']
print('各 Segment 的平均值：')
for cluster in range(num_clusters):
    print(f'Segment {cluster}:')
    print(f'Frequency 平均值: {frequency_mean[cluster]}')
    print(f'Recently 平均值: {recently_mean[cluster]}')
    print(f'Monetary 平均值: {monetary_mean[cluster]}')



RFM Model EDMConsent

In [ ]:
basic_df_edm = basic_df.merge(sa_all[['Cus_id','EDMSubscriber']], on='Cus_id', how='left')
basic_df_edm = basic_df_edm.drop_duplicates(subset='Cus_id')
edm_consent = basic_df_edm[basic_df_edm['EDMSubscriber'] == 'yes'].groupby('Segment').size().reset_index(name='EDMConsentYesCount')
print(edm_consent)

RFM Model Product

In [ ]:
sa_registered_edmconsent_segment = sa_registered_edmconsent.merge(basic_df[['Cus_id', 'Segment']], on='Cus_id', how='inner')
sa_registered_edmconsent_segment_product = basic_df[['Cus_id','Frequency', 'Recently', 'Monetary', 'Segment']].merge(sa_registered_edmconsent_segment[['Cus_id', 'product_line', 'p1', 'p2', 'p3', 'series','item','register_date','ebs_price']], on='Cus_id', how='inner')

cluster_product_line_count = sa_registered_edmconsent_segment_product.groupby(['Segment', 'product_line']).size().reset_index(name='Count')
cluster_total_counts = cluster_product_line_count.groupby('Segment')['Count'].sum()
cluster_product_line_count = cluster_product_line_count.merge(cluster_total_counts, on='Segment', suffixes=('', '_total'))
cluster_product_line_count['Percentage'] = (cluster_product_line_count['Count'] / cluster_product_line_count['Count_total']) * 100
cluster_product_line_count_sorted = cluster_product_line_count.sort_values(['Segment', 'Percentage'], ascending=[True, False])
top5_product_lines_per_cluster = cluster_product_line_count_sorted.groupby('Segment').head(5)
avg_monetary_per_cluster = top5_product_lines_per_cluster.merge(sa_registered_edmconsent_segment_product, on=['Segment', 'product_line'], how='left') \
                                                         .groupby(['Segment', 'product_line'])['ebs_price'].mean() \
                                                         .reset_index(name='Average_Price')
print('每個群中 product_line 頻率百分比前五高的 value 以及其平均 Monetary：')
print(top5_product_lines_per_cluster.merge(avg_monetary_per_cluster, on=['Segment', 'product_line']))

In [ ]:
p3_cluster_counts = sa_registered_edmconsent_segment_product.groupby(['Segment', 'p3']).size().reset_index(name='Count')
pivot_p3_cluster = p3_cluster_counts.pivot_table(index='Segment', columns='p3', values='Count', fill_value=0)
for segment, row in pivot_p3_cluster.iterrows():
    print(f"Segment {segment}:")
    nonzero_p3 = row[row > 0]
    if not nonzero_p3.empty:
        total_count = nonzero_p3.sum()
        p3_proportions = nonzero_p3 / total_count
        top_p3 = p3_proportions.sort_values(ascending=False).head(5)
        for p3_name, proportion in top_p3.items():
            print(f"  p3: {p3_name}, Count: {nonzero_p3[p3_name]}, Proportion: {proportion * 100:.2f}%")
    else:
        print("  No p3 with nonzero count.")
    print("\n")



In [ ]:
cols = ['product_line', 'p1', 'p2', 'p3','PL23','PL23S']
col = cols[0]
onehot_df = pd.get_dummies(sa_registered_edmconsent_segment_product[col], prefix='')
grouped_df = pd.concat([sa_registered_edmconsent_segment_product['Cus_id'], onehot_df], axis=1).groupby('Cus_id').sum()
for name in grouped_df.columns:
    if name.startswith('_'):
        grouped_df[f"{name}{name}{name}"] = (grouped_df[name] == 3).astype(int)
        grouped_df[f"{name}{name}"] = (grouped_df[name] >= 2).astype(int)
        grouped_df[name] = (grouped_df[name] >= 1).astype(int)
grouped_df = grouped_df.apply(lambda x: (x > 0).astype(int)).reset_index()
basic_cluster = basic_df[['Cus_id', 'Segment', 'Frequency']]
basic_cluster = basic_cluster.set_index('Cus_id')
product_combination = grouped_df.merge(basic_cluster, left_on='Cus_id', right_index=True, how='left')

In [ ]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
for Segment in range(4):
    group_df = product_combination[(product_combination['Segment'] == Segment) & (product_combination['Frequency'] >= 2)].drop(columns=['Segment', 'Frequency'])
    transactions = []
    for _, row in group_df.iterrows():
        items = set(group_df.columns[row == 1])
        transactions.append(items)

    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    group_fp_growth_df = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = fpgrowth(
        group_fp_growth_df, min_support=0.02, use_colnames=True
        )
    frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
    print(f"Group {Segment} - Frequent Itemsets:")
    print(frequent_itemsets)
    print("\n")

EDM Target

In [ ]:
basic_edm_target = sa_all[['Cus_id','EDM_Target']].merge(basic_df[['Cus_id', 'Segment']], on='Cus_id', how='left')
basic_edm_target = basic_edm_target.drop_duplicates(subset='Cus_id')

EDM_Target_yes = basic_edm_target[basic_edm_target['EDM_Target'] == 'yes'].groupby('Segment')
for segment, group in EDM_Target_yes:
    print(segment)
    print(len(group))

print(basic_edm_target)
print(basic_edm_target[(basic_edm_target['EDM_Target'] == 'yes') & (basic_edm_target['Segment'] != 0) & (basic_edm_target['Segment'] != 1) & (basic_edm_target['Segment'] != 2) & (basic_edm_target['Segment'] != 3)])

RFM Model Product(EDM consent isn't considered)

In [ ]:
basic_df_product = basic_df[['Cus_id','Frequency', 'Recently', 'Monetary', 'Segment']].merge(sa_all[['Cus_id', 'product_line']], on='Cus_id', how='left')
basic_df_product_inner = basic_df[['Cus_id','Frequency', 'Recently', 'Monetary', 'Segment']].merge(sa_all[['Cus_id', 'product_line','register_date','ebs_price']], on='Cus_id', how='inner')
basic_df_product_detail = basic_df[['Cus_id','Frequency', 'Recently', 'Monetary', 'Segment']].merge(sa_all[['Cus_id', 'product_line', 'p1', 'p2', 'p3', 'series','item']], on='Cus_id', how='inner')

cluster_product_line_count = basic_df_product.groupby(['Segment', 'product_line']).size().reset_index(name='Count')
cluster_total_counts = cluster_product_line_count.groupby('Segment')['Count'].sum()
cluster_product_line_count = cluster_product_line_count.merge(cluster_total_counts, on='Segment', suffixes=('', '_total'))
cluster_product_line_count['Percentage'] = (cluster_product_line_count['Count'] / cluster_product_line_count['Count_total']) * 100
cluster_product_line_count_sorted = cluster_product_line_count.sort_values(['Segment', 'Percentage'], ascending=[True, False])
top5_product_lines_per_cluster = cluster_product_line_count_sorted.groupby('Segment').head(5)
avg_monetary_per_cluster = top5_product_lines_per_cluster.merge(basic_df_product_inner, on=['Segment', 'product_line'], how='left') \
                                                         .groupby(['Segment', 'product_line'])['ebs_price'].mean() \
                                                         .reset_index(name='Average_Price')
print('每個群中 product_line 頻率百分比前五高的 value 以及其平均 Monetary：')
print(top5_product_lines_per_cluster.merge(avg_monetary_per_cluster, on=['Segment', 'product_line']))

In [ ]:
p3_cluster_counts = basic_df_product_detail.groupby(['Segment', 'p3']).size().reset_index(name='Count')
pivot_p3_cluster = p3_cluster_counts.pivot_table(index='Segment', columns='p3', values='Count', fill_value=0)
for segment, row in pivot_p3_cluster.iterrows():
    print(f"Segment {segment}:")
    # 去除計數為 0 的 p3
    nonzero_p3 = row[row > 0]
    # 如果有非零的 p3，則計算比例並輸出前五個
    if not nonzero_p3.empty:
        total_count = nonzero_p3.sum()
        p3_proportions = nonzero_p3 / total_count
        top_p3 = p3_proportions.sort_values(ascending=False).head(5)
        for p3_name, proportion in top_p3.items():
            print(f"  p3: {p3_name}, Count: {nonzero_p3[p3_name]}, Proportion: {proportion * 100:.2f}%")
    else:
        print("  No p3 with nonzero count.")
    print("\n")

In [ ]:
cols = ['product_line', 'p1', 'p2', 'p3','PL23','PL23S']
col = cols[0]
onehot_df = pd.get_dummies(basic_df_product_detail[col], prefix='')
grouped_df = pd.concat([basic_df_product_detail['Cus_id'], onehot_df], axis=1).groupby('Cus_id').sum()
for name in grouped_df.columns:
    if name.startswith('_'):
        grouped_df[f"{name}{name}{name}"] = (grouped_df[name] == 3).astype(int)
        grouped_df[f"{name}{name}"] = (grouped_df[name] >= 2).astype(int)
        grouped_df[name] = (grouped_df[name] >= 1).astype(int)
grouped_df = grouped_df.apply(lambda x: (x > 0).astype(int)).reset_index()
basic_cluster = basic_df[['Cus_id', 'Segment', 'Frequency']]
basic_cluster = basic_cluster.set_index('Cus_id')
product_combination = grouped_df.merge(basic_cluster, left_on='Cus_id', right_index=True, how='left')

In [ ]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
for Segment in range(4):
    group_df = product_combination[(product_combination['Segment'] == Segment) & (product_combination['Frequency'] >= 2)].drop(columns=['Segment', 'Frequency'])
    transactions = []
    for _, row in group_df.iterrows():
        items = set(group_df.columns[row == 1])
        transactions.append(items)

    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    group_fp_growth_df = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = fpgrowth(
        group_fp_growth_df, min_support=0.05, use_colnames=True
        )
    frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
    print(f"Group {Segment} - Frequent Itemsets:")
    print(frequent_itemsets)
    print("\n")

匯入Bigquery Table

In [ ]:
sa_all_segment = sa_all.merge(basic_df[['Cus_id', 'Segment']], on='Cus_id', how='left')
BigQuery_table = sa_all_segment[['Cus_id', 'Segment']].drop_duplicates(subset=['Cus_id'])

BigQuery_table = BigQuery_table.rename(columns={'Cus_id': 'member_id', 'Segment': 'segment'})
BigQuery_table['segment'] = BigQuery_table['segment'].replace({
    2: 'RFM_VIP',
    1: 'RFM_Hibernating',
    3: 'RFM_New',
    0: 'RFM_Need Attention',
})
BigQuery_table['segment'] = BigQuery_table['segment'].fillna('RFM_Unregistered')
print(BigQuery_table)

BigQuery_table.to_csv('BigQuery_table.csv', index=False)
single_sa_all_EDM = sa_all[sa_all['EDMSubscriber'] == 'yes']
BigQuery_table_2 = single_sa_all_EDM.merge(basic_df[['Cus_id', 'Segment']], on='Cus_id', how='left')
BigQuery_table_2 = BigQuery_table_2[['Cus_id', 'Segment']]
BigQuery_table_2 = BigQuery_table_2.rename(columns={'Cus_id': 'member_id', 'Segment': 'segment'})
BigQuery_table_2['segment'] = BigQuery_table_2['segment'].replace({
    2: 'RFM_VIP',
    1: 'RFM_Hibernating_1',
    3: 'RFM_New',
    0: 'RFM_Hibernating_2',
})
BigQuery_table_2['segment'] = BigQuery_table_2['segment'].fillna('RFM_Unregistered')
BigQuery_table_2 = BigQuery_table_2.drop_duplicates(subset=['member_id'])
print(BigQuery_table_2)
BigQuery_table_2.to_csv('BigQuery_table_3.csv', index=False)
filtered_data = basic_df_product_inner[(basic_df_product_inner['Segment'] == 1) & (basic_df_product_inner['product_line'].isin(['NR', 'NB']))]
filtered_data = filtered_data[['Cus_id', 'product_line', 'register_date']]
filtered_data['register_date'] = pd.to_datetime(filtered_data['register_date'])

nb_data = filtered_data[filtered_data['product_line'] == 'NB']
nr_data = filtered_data[filtered_data['product_line'] == 'NR']
result_df = pd.DataFrame()
for cus_id in filtered_data['Cus_id'].unique():
    nb_date = nb_data[nb_data['Cus_id'] == cus_id]['register_date'].max()
    nr_date = nr_data[nr_data['Cus_id'] == cus_id]['register_date'].max()

    if not pd.isna(nb_date) and not pd.isna(nr_date):
        period = (nb_date - nr_date).days
        result_df = pd.concat([result_df, pd.DataFrame({'Cus_id': [cus_id], 'period': [period]})], ignore_index=True)

abs_period_values = result_df['period'].abs()
average_abs_period = 1.5*abs_period_values.mean()
print("1.5倍絕對值平均:", average_abs_period)